#Chapter 3: It Starts with a Tensor
*   Tensors, the basic data structure in PyTorch
*   Indexing and operating on PyTorch tensors to explore and manipulate data
*   Interoperating with NumPy multidimensional arrays
*   Moving computations to the GPU for speed

Through this chapter, we are going to understand how PyTorch handles and stores data as input, as intermediate representations, and as output.

In [ ]:
a=[1.0,2.0,1.0]

In [ ]:
a[0]

1.0

In [ ]:
a[2]

1.0

Constructing first tensor

In [ ]:
import torch #<1>
a=torch.ones(3) #<2>
a

tensor([1., 1., 1.])

In [ ]:
a[1]

tensor(1.)

In [ ]:
float(a[1])

1.0

In [ ]:
a[2]=2.0
a

tensor([1., 1., 2.])

In [ ]:
points=torch.zeros(6) #<1> The use of .zeros here is just a way to get an appropiately sized array
points[0]=4.0 #<2> We overwrite those zeros with the values we actually want
points[1]=1.0
points[2]=5.0
points[3]=3.0
points[4]=2.0
points[5]=1.0
points

tensor([4., 1., 5., 3., 2., 1.])

In [ ]:
float(points[0]),float(points[1])

(4.0, 1.0)

In [ ]:
points=torch.tensor([[4.0,1.0],[5.0,3.0],[2.0,1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [ ]:
points.shape

torch.Size([3, 2])

In [ ]:
#We can have used zeros or ones to initialize the tensor, providing the size as a tuple
points=torch.zeros(3,2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [ ]:
#We can access individual element in the tensor using two indices:
points[1,1]

tensor(3.)

In [ ]:
points[1]

tensor([5., 3.])

Indexing Tensor

In [ ]:
some_list=list(range(6))
some_list[:]     #All elements in the list
some_list[1:4]   #From element 1 inclusive to element 4 exclusive
some_list[1:]    #From element 1 inclusive to end of the list
some_list[:4]    #From the start of the list to element 4 exclusive
some_list[:-1]   #From the start of the list to one before the last element
some_list[1:4:2] #From element 1 inclusive to element 4 exclusive in steps of 2

In [ ]:
#points[1:]   #All rows after first, implicitly all columns
#points[1:,:] #All rows after first, all columns
#points[1:,0] #All rows after first, first column
points[None] #Add dimension of size one, just like unsqueeze

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

Named Tensors

In [ ]:
img_t=torch.randn(3,5,5) #Shape [channels (RGB), rows, columns]
weights=torch.tensor([0.2126,0.7152,0.0722])

In [ ]:
batch_t=torch.randn(2,3,5,5) #Shape [batch,channels,rows,columns]

In [ ]:
img_gray_naive=img_t.mean(-3)
batch_gray_naive=batch_t.mean(-3)
img_gray_naive.shape,batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [ ]:
unsqueezed_weights=weights.unsqueeze(-1).unsqueeze_(-1)
img_weights=(img_t*unsqueezed_weights)
batch_weights=(batch_t*unsqueezed_weights)
img_gray_weighted=img_weights.sum(-3)
batch_weights.shape,batch_t.shape,unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [ ]:
img_gray_weighted_fancy=torch.einsum('...chw,c->...hw',img_t,weights)
batch_gray_weighted_fancy=torch.einsum('...chw,c->...hw',batch_t,weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [ ]:
weights_named=torch.tensor([0.2126, 0.7152, 0.0722],names=['channels'])
weights_named

/pytorch/c10/core/TensorImpl.h:806: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable.


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [ ]:
img_named=img_t.refine_names(...,'channels','rows','columns')
batch_named=batch_t.refine_names(...,'channels','rows','columns')
print("img named:",img_named.shape,img_named.names)
print("batch named:",batch_named.shape,batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [ ]:
weights_aligned=weights_named.align_as(img_named)
weights_aligned.shape,weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [ ]:
gray_named=(img_named*weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [ ]:
gray_plain=gray_named.rename(None)
gray_plain.shape,gray_plain.names

(torch.Size([5, 5]), (None, None))